In [ ]:
import numpy as np
import pandas as pd
import io, requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.relative_locator import locate_with
import time

In [ ]:
cdata = pd.read_csv('Cleaned Data')

In [ ]:
cdata.shape

In [ ]:
cdata.isna().sum()

In [ ]:
c = 0
l = []
for u in cdata.name.isna():
    if u == True:
        l.append(c)
    c+=1

In [ ]:
cdata.drop(l,axis=0,inplace=True)

In [ ]:
counter = 16000

In [ ]:
name = cdata.name[counter+1:counter+2000]
ffx = webdriver.Chrome()

ffx.get("https://www.transfermarkt.com")

bigLi = []
notfound = []

for e in name:

    li = []
    li.append(e) 
    try:
        # Search with player name
        ffx.find_element(By.CSS_SELECTOR, '[class="tm-header__input--search-field"]').send_keys(e) 


        # press Enter Key
        ffx.find_element(By.CSS_SELECTOR,'[class="tm-header__input--search-send"]').send_keys(Keys.ENTER) 

        player = ffx.find_element(By.CSS_SELECTOR,'td[class= "rechts hauptlink"]')
        li3.append(player.text)
        
        # get player profile link like :   " https://www.transfermarkt.com/james-rodriguez/profile/spieler/88103 "
        url = ffx.find_element(By.CSS_SELECTOR,'td[class= "hauptlink"]').find_element(By.TAG_NAME,'a').get_attribute('href')
        
    except:
        ffx.find_element(By.CSS_SELECTOR, '[class="tm-header__input--search-field"]').clear()
        notfound.append(counter)
        continue
    counter +=1
    pages = ['leistungsdatenverein', 'nationalmannschaft', 'marktwertverlauf']

                #leistungsdatenverein --->  stat page
                #nationalmannschaft  --->  natoinal team stat page
                #marktwertverlauf   ---> markert value rank

    for page in pages:                    # switching betweem website pages from change this part  ----/profile/----
        x = url.split('/')
        x[4] = page
        for i in range(len(x)):
            x[i] = x[i] + "/"
        url = ''.join(str(u) for u in x)

        ffx.get(url)



        if page == 'leistungsdatenverein':  
            try:
                stat = ffx.find_element(By.CSS_SELECTOR, 'div[id="yw1"]').find_element(By.TAG_NAME,'tfoot')  
                
                for i, j in zip(stat.find_elements(By.CSS_SELECTOR, 'td[class="zentriert"]'), range(1,7)):
                    li.append(i.get_attribute("innerHTML"))

                for x, j in zip(stat.find_elements(By.CSS_SELECTOR, 'td[class="rechts"]'), range(7,9)):
                    li.insert(j, x.get_attribute("innerHTML"))
            except:
                while len(li) < 9 :
                    li.append('none')


        elif page == 'nationalmannschaft':  
            try:
                stat = ffx.find_element(By.CSS_SELECTOR, 'div[id="yw1"]').find_element(By.TAG_NAME,'tfoot')
                
                for i, j in zip(stat.find_elements(By.CSS_SELECTOR, 'td[class="zentriert"]'), range(9,15)):
                    li.insert(j, i.get_attribute("innerHTML"))

                for x, j in zip(stat.find_elements(By.CSS_SELECTOR, 'td[class="rechts"]'), range(15,17)):
                    li.insert(j, x.get_attribute("innerHTML"))
            except:
                while len(li) < 17 :
                    li.append('none')

        else:
            try:
                for i, j in zip(ffx.find_elements(By.CSS_SELECTOR, 'span[class="quick-fact__content--large"]'), range(17,23)):
                    li.append(i.get_attribute("innerHTML"))
            except :   
                while len(li) < 23 :
                    li.append('none')
    ffx.find_element(By.CSS_SELECTOR, '[class="tm-header__input--search-field"]').clear()
    bigLi.append(li)


ffx.quit()

In [ ]:
counter

In [ ]:
notfound

In [ ]:
q = 0
for z, i  in zip(bigLi, range(len(bigLi))):
    while len(z) < 22 :
        z.append('none')
    else :
        bigLi[i] = bigLi[i][:22]
    q+=1

In [ ]:
df = pd.DataFrame(bigLi, columns=['names', 'apperance', 'goals', 'assists', 'yellow_card', '2nd_ycard', 'red_card', 'drop','min', 'apperance_NT', 'goals_NT', 'assists_NT', 'yellow_card_NT', '2nd_ycard_NT', 'red_card_NT', 'drop NT','min_NT','world_rank', 'league_rank', 'club_rank', 'National_rank', 'position_rank'])                                     

In [ ]:
df.drop(['drop', 'drop NT'], axis= 1, inplace=True)

In [ ]:
data = pd.read_csv('2ndscraped_data')

In [ ]:
data.drop(data.columns[0], axis=1, inplace= True)

In [ ]:
df.replace('-',0, inplace=True)

In [ ]:
data = data.append(df, ignore_index=True)

In [ ]:
data.to_csv('2ndscraped_data')